In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

import os
from pandas import DataFrame
from notebook_utils import NotebookUtilities

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))


# OSU dataset of Simulation Sessions, deidentified

Doug screened all of the **VOICE_COMMAND** and **VOICE_CAPTURE** lines and replaced any names with either Max or Jane, regardless of whether the name was that of the responder.

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    display(df.T)

(149166, 100)


,13546,125352,2356,84694
action_type,TAG_APPLIED,VOICE_CAPTURE,S_A_L_T_WAVE_IF_CAN,TOOL_HOVER
action_tick,1881829,85225,1776818,113179
event_time,2023-05-16 16:46:00,2023-05-31 09:46:00,2023-05-16 15:28:00,2023-05-09 08:54:00
session_uuid,724fdf45-3165-43b8-b9ca-dc07102d2886,e96d123b-455b-4335-9278-988ff9ba8ecf,06574b6f-ab02-432c-9a65-7b031218a270,8f6097ce-795d-483a-a811-e6a5423bc59d
file_name,724fdf45-3165-43b8-b9ca-dc07102d2886,e96d123b-455b-4335-9278-988ff9ba8ecf,06574b6f-ab02-432c-9a65-7b031218a270,8f6097ce-795d-483a-a811-e6a5423bc59d
scene_index,6,1,6,1
voice_capture_message,NaN,yeah it's weird,NaN,NaN
s_a_l_t_wave_if_can_sort_location,NaN,NaN,"(2.9, 0.0, 0.1)",NaN
s_a_l_t_wave_if_can_sort_command_text,NaN,NaN,waveIfCan,NaN
s_a_l_t_wave_if_can_patient_id,NaN,NaN,Mike_1 Root,NaN



## Decision Points

<p>
    OSU and Big Bear: With the IRB approval (for the previous OSU dataset) in place, I would like to get working on the dataset asap. That means:
    <ul>
        <li>Getting it deidentified and sharable</li>
        <li>If there is general demographic information like gender or level of expertise that can be associated with the VR performance, that would be useful.</li>
        <li>Big Bear: once it is available to you, I want an exploratory analysis on decision points such that I can see the variance in responses to the same situation.</li>
        <li>For example, if there are 3 waving patients and the participant chooses to move toward one and assess them, I want to know the sim state (visual info. on each patient, distance, etc.) and then a breakdown of how many participants chose to move toward each patient.</li>
        <li>Similar for treatment options, for each patient present in the simulated environment, what is the variance in which treatment is applied</li>
         <li>Does it depend on ordr?</li>
          <li>Ting?</li>
           <li>Be creative and ask all the questions. The goal is to provide TA1 with this dataset if there is anything useful to be found in analyzing the decisions (not necessarily that we find results but that the data is in shape to ask these questions). Let me know if you need additional clarifiion   </l    i>
</</p>
ion.

In [4]:

# List the action types
frvrs_logs_df.action_type.value_counts().sort_index()

action_type
BAG_ACCESS              2180
BAG_CLOSED              2149
INJURY_RECORD           3052
INJURY_TREATED          1577
PATIENT_DEMOTED         1818
PATIENT_ENGAGED         2295
PATIENT_RECORD          2063
PULSE_TAKEN             3427
SESSION_END              256
SESSION_START            266
S_A_L_T_WALKED          2725
S_A_L_T_WALK_IF_CAN     4567
S_A_L_T_WAVED           1761
S_A_L_T_WAVE_IF_CAN     2147
TAG_APPLIED             1819
TAG_DISCARDED            533
TAG_SELECTED            2353
TELEPORT                5678
TOOL_APPLIED            1818
TOOL_DISCARDED          1655
TOOL_HOVER             89733
TOOL_SELECTED           2263
VOICE_CAPTURE          10115
VOICE_COMMAND           2916
Name: count, dtype: int64


These action types mean that the DM has made a decision: INJURY_TREATED, PATIENT_ENGAGED, PULSE_TAKEN, TAG_APPLIED, and TOOL_APPLIED.

In [26]:

# Get the last previous tool hover for each patient engaged
if nu.pickle_exists('hover_engage_df'):
    hover_engage_df = nu.load_object('hover_engage_df')
else:
    import numpy as np
    
    rows_list = []
    for (session_uuid, scene_index), df in fu.get_session_groupby(mask_series=None, extra_column='scene_index'):
        hover_mask_series = (df.action_type == 'TOOL_HOVER')
        for row_index, row_series in df[hover_mask_series].iterrows():
            hover_elapsed_time = row_series.action_tick
            engage_mask_series = (df.action_tick >= hover_elapsed_time) & ~df.patient_id.isnull()
            if df[engage_mask_series].shape[0]:
    
                # Get the patient's ID
                engage_elapsed_time = df[engage_mask_series].action_tick.min()
                patient_mask_series = (df.action_tick == engage_elapsed_time) & ~df.patient_id.isnull()
                patient_id = df[patient_mask_series].iloc[0].patient_id
    
                # Get the required procedure
                require_mask_series = (df.patient_id == patient_id) & ~df.injury_record_required_procedure.isnull()
                if df[require_mask_series].shape[0]:
                    injury_record_required_procedure = nu.conjunctify_nouns(df[require_mask_series].injury_record_required_procedure.tolist())
                else:
                    injury_record_required_procedure = np.nan
    
                # Get what tool the DM is looking at
                tool_hover_type = row_series.tool_hover_type
    
                # Add that info to the data frame
                row_dict = {
                    'session_uuid': session_uuid, 'scene_index': scene_index, 'tool_hover_type': tool_hover_type,
                    'injury_record_required_procedure': injury_record_required_procedure, 'patient_id': patient_id
                }
                rows_list.append(row_dict)
    
    
    
    hover_engage_df = DataFrame(rows_list).drop_duplicates(subset=fu.patient_groupby_columns, keep='last')
    
    # Get the last previous tool hover for each patient engaged
    def f(row_series):
        tool_hover_type = str(row_series.tool_hover_type)
        injury_record_required_procedure = str(row_series.injury_record_required_procedure)
        
        if (tool_hover_type.lower() in injury_record_required_procedure):
            similarity = 1.0
        elif (tool_hover_type == 'Tourniquet') and ('tourniquet' not in injury_record_required_procedure):
            similarity = 0.0
        elif (tool_hover_type == 'Hemostatic Gauze') and ('woundpack' in injury_record_required_procedure):
            similarity = 1.0
        elif (tool_hover_type == 'Hemostatic Gauze') and ('woundpack' not in injury_record_required_procedure):
            similarity = 0.0
        elif (tool_hover_type == 'Gauze') and ('gauzePressure' in injury_record_required_procedure):
            similarity = 1.0
        elif (tool_hover_type == 'Gauze') and ('gauzePressure' not in injury_record_required_procedure):
            similarity = 0.0
        elif (tool_hover_type == 'Needle') and ('airway' in injury_record_required_procedure):
            similarity = 1.0
        elif (tool_hover_type == 'Needle') and ('airway' not in injury_record_required_procedure):
            similarity = 0.0
        elif (tool_hover_type != 'NaN') and (injury_record_required_procedure in ['none', 'NaN']):
            similarity = 0.0
        elif (injury_record_required_procedure in ['none', 'NaN']):
            similarity = 1.0
        else:
            similarity = nu.compute_similarity(tool_hover_type, injury_record_required_procedure)
        
        return similarity
    hover_engage_df['similarity'] = hover_engage_df.apply(f, axis='columns')

    # Store the data frame as a pickle
    nu.store_objects(hover_engage_df=hover_engage_df)

In [42]:

percent_accuracy = 100*hover_engage_df.similarity.sum()/hover_engage_df.similarity.shape[0]
print(f'The last previous tool hover correctly anticipates the required injury procedure for each patient engaged {percent_accuracy:.1f}% of the time.')

The last previous tool hover correctly anticipates the required injury procedure for each patient engaged 65.5% of the time.
